# Simulation of infection spreads in a network

This demo will run a simulation in how an infection can spread in a small network.

Initial idea: model the people as a graph. This limits the direct neighbors to 8, but we will change this assumption later on. Also, populate the matrix randomly.

Couple of assumptions:
1. Once a node is infected, it is cured after 14 days, unless it dies.
2. Once a node is cured after being infected, it has immunity to infection till 90 days.
3. New nodes can be added to the network randomly, with probability 0.01.
4. Once infected, a node has 0.03 probability of death. To simplify things, I will assume death happens in 14 days as well.
5. Nodes don't move.
6. Every turn of the simulation marks one day.

In [12]:
import numpy as np
import matplotlib.pyplot as plot
from networkx.generators.random_graphs import erdos_renyi_graph
import networkx as nx
from enum import Enum
%matplotlib inline
import ipycytoscape

import time

I use `ipycytoscape` as my visualization library, which will allow me to interact with graphs directly

In [2]:
import ipywidgets as widgets

In [3]:
from typing import Callable, Iterator, Union, Optional, List

I keep a `Status` enum to denote node health

In [4]:
class Status:
    HEALTHY = 'HEALTHY'
    IMMUNE = 'IMMUNE'
    INFECTED = 'INFECTED'
    DEAD = 'DEAD'


For my `Node` class, I keep a simple structure. I keep track of the current "health" of the node, and keep two counters to keep track of infection and immunity time. 

In [5]:
class MyNode(ipycytoscape.Node):
    MAX_IMMUNITY_TIME = 4
    MAX_INFECTED_TIME = 2
    PROB_DEATH = 0
    def __init__(self,
                 nodeNum: int,
                 maxImmunityTime: int, 
                 maxInfectionTime: int,
                 probDeath: int,
                 status: Status = Status.HEALTHY):
        super().__init__()
        self.data['id'] = nodeNum
        self.maxImmunityTime = maxImmunityTime
        self.maxInfectionTime = maxInfectionTime
        self.probDeath = probDeath
        self.classes = status
        self.infectionTime = 0
        self.immunityTime = 0
        self.willDie = 0
        
    def __str__(self):
        temp = super().__str__()
        return f"super: {temp}, infectionTime: {self.infectionTime}, immunityTime: {self.immunityTime}, willDie: {self.willDie}"
    
    def processCycle(self, time: int, infect: bool) -> bool:
        if self.classes == Status.DEAD:
            return True
        if infect:
            if self.classes == Status.HEALTHY:
                self.infect(time)
            return False
        
        if self.classes == Status.IMMUNE:
            if time - self.immunityTime > self.maxImmunityTime:
                self.disinfect()
            return False
                    
        if self.classes == Status.INFECTED:
                if time - self.infectionTime > self.maxInfectionTime:
                    if self.willDie:
                        self.classes = Status.DEAD
                        return True
                    self.immune(time)
        return False
    
    def immune(self, time: int):
        self.classes = Status.IMMUNE
        self.immunityTime = time
        self.infectionTime = 0

    
    def disinfect(self):
        self.classes = Status.HEALTHY
        self.infectionTime = 0
        self.immunityTime = 0

        
    def infect(self, infected_time: int):
        self.infectionTime = infected_time
        self.classes = Status.INFECTED
        self.immunityTime = 0
        self.willDie = np.random.choice([1,0], 1, p=[self.probDeath, 1-self.probDeath])[0]


Finally, I create my network. Healthy nodes are indicated in blue, while infected are in red, and immune are in green.

In [7]:
class Network:
    def __init__(self, numNodes: int = 200, 
                 probEdges: int = 2, 
                 infectionTime = MyNode.MAX_INFECTED_TIME, 
                 immunityTime = MyNode.MAX_IMMUNITY_TIME,
                 probInfection = 0.2):
        self.probInfection = probInfection
        self.time = 0
        self.numNodes = numNodes
        self.infectedNodes = set()
            
        
        mapping = [MyNode(nodeNum = i, maxImmunityTime = immunityTime,
                                   maxInfectionTime = infectionTime, probDeath=0) for i in range(numNodes)]
        sample_graph = nx.barabasi_albert_graph(numNodes, probEdges)
        self.graph = nx.Graph()
        for i in range(numNodes):
            self.graph.add_node(mapping[i])
        for edge in sample_graph.edges:
            left = edge[0]
            right = edge[1]
            self.graph.add_edge(mapping[left], mapping[right])
        
            
    def initializeInfected(self, infected: float = 0.05):
        prob = np.random.choice([0,1], self.numNodes, p=[infected, 1-infected])
        self.infectedNodes = set()
        i = 0
        for node in self.graph.nodes:
            if prob[i] == 0:
                node.infect(0)
                self.infectedNodes.add(node)
            else:
                node.disinfect()
            i += 1
        self.time = 0
            
            
    def neighbor(self, i: int) -> List[MyNode]:
        return self.graph.neighbors(i)
    
    def simulateOneCycle(self):
        self.time = self.time+1
        
        # first gather the list of nodes which will be infected this cycle
        to_be_infected_nodes = set()
        for i in self.infectedNodes:
            numNeighbor = 0
            for x in self.neighbor(i):
                numNeighbor += 1
            probInfection = np.random.choice([1,0], numNeighbor, 
                                             p=[self.probInfection, 1 - self.probInfection])
            
            j = 0
            for neighbor in self.neighbor(i):
                if probInfection[j]:
                    to_be_infected_nodes.add(neighbor) # add probability here
                j += 1

                
        self.infectedNodes = set()
        # Now, run the simulation for all nodes, keeping track of newly infected nodes
        for node in self.graph.nodes:
            if node in to_be_infected_nodes:
                node.processCycle(self.time, True)
            else:
                node.processCycle(self.time, False)
            if node.classes == Status.INFECTED:
                self.infectedNodes.add(node)
        if len(self.infectedNodes) == 0:
            return True
        return False
    
    def simulate(self, infected: float = None, delay: int = 500, days: int = 50):
        if infected is not None: 
            x.initializeInfected(infected = infected)
        for i in range(days):
            time.sleep(delay/1000)
            if self.simulateOneCycle():
                return i
                break
        return days
        
                

In [8]:
x = Network(100, probEdges=2, infectionTime=4, immunityTime=6, probInfection=0.1)
x.initializeInfected(infected = 0.2)


In [9]:
widget = ipycytoscape.CytoscapeWidget()
widget.set_layout(nodeSpacing=10, animation=False)
widget.graph.add_graph_from_networkx(x.graph)
widget.set_style([
                        {
                            'selector': 'node.HEALTHY',
                            'css': {
                                'background-color': 'blue'
                            }
                        },
                        {
                            'selector': 'node.INFECTED',
                            'css': {
                                'background-color': 'red'
                            }
                        },
                        {
                            'selector': 'node.IMMUNE',
                            'css': {
                                'background-color': 'green'
                            }
                        }])

In [10]:
display(widget)

CytoscapeWidget(cytoscape_layout={'name': 'cola', 'nodeSpacing': 10, 'animation': False}, cytoscape_style=[{'s…

In [13]:
i = x.simulate(days=150)

In [14]:
i

126

Now, I want to model some more things:
1. The probability of someone dying (or leaving this network)
2. The probability of birth (or someone new arriving in this network)

For simplicity of model, I will assume that each time a new node is added to the network, it is healthy.

In [26]:
class Network2(Network):
    
    def __init__(self, numNodes: int = 200, 
                 probEdges: float = 0.02, 
                 infectionTime = MyNode.MAX_INFECTED_TIME, 
                 immunityTime = MyNode.MAX_IMMUNITY_TIME,
                 probDeath = 0.2,
                 probBirth = 0.2,
                 probInfection = 0.2):
        self.probInfection = probInfection
        self.time = 0
        self.numNodes = numNodes
        self.infectedNodes = set()
            
        
        mapping = [MyNode(nodeNum = i, maxImmunityTime = immunityTime,
                                   maxInfectionTime = infectionTime, probDeath=probDeath) for i in range(numNodes)]
        sample_graph = nx.barabasi_albert_graph(numNodes, probEdges)
        self.graph = nx.Graph()
        for i in range(numNodes):
            self.graph.add_node(mapping[i])
        for edge in sample_graph.edges:
            left = edge[0]
            right = edge[1]
            self.graph.add_edge(mapping[left], mapping[right])
        
        
    def simulateOneCycle(self, widget):
        self.time = self.time+1
        
        # first gather the list of nodes which will be infected this cycle
        to_be_infected_nodes = set()
        for i in self.infectedNodes:
            numNeighbor = 0
            for x in self.neighbor(i):
                numNeighbor += 1
            probInfection = np.random.choice([1,0], numNeighbor, 
                                             p=[self.probInfection, 1 - self.probInfection])
            
            j = 0
            for neighbor in self.neighbor(i):
                if probInfection[j]:
                    to_be_infected_nodes.add(neighbor) # add probability here
                j += 1

                
        self.infectedNodes = set()
        # Now, run the simulation for all nodes, keeping track of newly infected nodes and removing and adding nodes
        deadNodes = []
        for node in self.graph.nodes:
            isDead = node.processCycle(self.time, node in to_be_infected_nodes)
            if isDead:
                deadNodes.append(node)
            if node.classes is Status.INFECTED and not isDead:
                self.infectedNodes.add(node)
        
        # remove the dead nodes now
        for node in deadNodes:
            print("Removing node", node.data["id"])
            self.graph.remove_node(node)
            widget.graph.remove_node(node)
        if len(self.infectedNodes) == 0:
            return True
        return False
    def simulate(self, widget, infected: float = None, delay: int = 500, days: int = 50):
        if infected is not None: 
            x.initializeInfected(infected = infected)
        for i in range(days):
            time.sleep(delay/1000)
            if self.simulateOneCycle(widget):
                return i
                break
        return days
            

In [60]:
x = Network2(100, probEdges=2, infectionTime=2, immunityTime=11, probInfection=0.3, probDeath=0.2, probBirth=0)
x.initializeInfected(infected = 0.1)
widget = ipycytoscape.CytoscapeWidget()
widget.set_layout(nodeSpacing=10, animate=False)
widget.graph.add_graph_from_networkx(x.graph)
widget.set_style([
                        {
                            'selector': 'node.HEALTHY',
                            'css': {
                                'background-color': 'blue'
                            }
                        },
                        {
                            'selector': 'node.INFECTED',
                            'css': {
                                'background-color': 'red'
                            }
                        },
                        {
                            'selector': 'node.IMMUNE',
                            'css': {
                                'background-color': 'green'
                            }
                        }])
display(widget)

CytoscapeWidget(cytoscape_layout={'name': 'cola', 'nodeSpacing': 10, 'animate': False}, cytoscape_style=[{'sel…

In [61]:
i = x.simulate(widget)

Removing node 47
Removing node 48
Removing node 72
Removing node 92
Removing node 60
Removing node 6
Removing node 71
Removing node 74
Removing node 3
Removing node 43
Removing node 63
Removing node 79
Removing node 85
Removing node 31
Removing node 29
Removing node 38
Removing node 62
Removing node 89
Removing node 19


In [62]:
len(x.graph.nodes.items())

81

In [63]:
i

13